In [3]:
# training xception model with keras
import keras
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
import pickle
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf


In [4]:
# preparing the data 

with open('train.pkl', 'rb') as f:
    data = pickle.load(f)

X = []
y = []
for i, row in data.iterrows():
    if i > 10000: # only use first 10000 images
        break
    try:
        x1, y1, x2, y2 = row['bbox'][0], row['bbox'][1], row['bbox'][2], row['bbox'][3]
        img = cv2.imread(row['paths'])[y1:y2, x1:x2] # crop image
        img = cv2.resize(img, (224, 224)) # resize to 224x224
        img = img / 255.0 # normalize
        img = np.array(img) 
        X.append(img)
        if row['first_emotion'] in [3,4,5,6,9]:
            y.append(row['first_emotion'])
        else:
            y.append(-1)
    except:
        continue
    
X = np.asarray(X)
y = np.asarray(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 50 extraneous bytes before marker 0xd9


In [5]:
X_train.shape

(6920, 224, 224, 3)

In [6]:
model_xception = Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))
# add a global spatial average pooling layer
x = model_xception.output
y = GlobalAveragePooling2D()(x)
z = Dense(224, activation='relu')(y)
predictions = Dense(6, activation='softmax')(z)
model = Model(inputs=model_xception.input, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['mse'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-20 21:35:21.991730: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-20 21:35:21.992207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(param_str):    
    run_id = str(int(time.time())) + param_str
    return os.path.join(root_logdir, run_id)
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
run_logdir = get_run_logdir("xception" + "_emotion_recognition")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
es = keras.callbacks.EarlyStopping(min_delta=1.0, patience=10, verbose=1)

In [8]:
# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[es, tensorboard_cb], batch_size=32)


Epoch 1/10


2022-07-20 21:35:52.684388: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-20 21:35:53.996379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


217/217 [==============================] - ETA: 0s - loss: 1.3491 - accuracy: 0.2692

2022-07-20 21:38:56.299389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


217/217 [==============================] - 195s 883ms/step - loss: 1.3491 - accuracy: 0.2692 - val_loss: 165.6531 - val_accuracy: 0.2616
Epoch 2/10
217/217 [==============================] - 192s 883ms/step - loss: 7.0175 - accuracy: 0.2530 - val_loss: 47.8553 - val_accuracy: 0.2920
Epoch 3/10
217/217 [==============================] - 192s 885ms/step - loss: 28.2943 - accuracy: 0.2451 - val_loss: 1660.7126 - val_accuracy: 0.1382
Epoch 4/10
217/217 [==============================] - 192s 883ms/step - loss: 68.1214 - accuracy: 0.2393 - val_loss: 161.1404 - val_accuracy: 0.2448
Epoch 5/10
217/217 [==============================] - 192s 885ms/step - loss: 156.5548 - accuracy: 0.2185 - val_loss: 681.2783 - val_accuracy: 0.2920
Epoch 6/10
217/217 [==============================] - 246s 1s/step - loss: 277.5961 - accuracy: 0.2269 - val_loss: 286.7272 - val_accuracy: 0.2448
Epoch 7/10
217/217 [==============================] - 200s 921ms/step - loss: 328.3426 - accuracy: 0.2257 - val_loss: 10

In [9]:
# tensorboard visualization
import tensorboard as tb


In [10]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

47/47 [==============================] - 12s 246ms/step - loss: 71.8569 - accuracy: 0.3048


In [11]:
# predict the test set
y_pred = model.predict(X_test)

2022-07-20 22:09:31.657974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [12]:
print(y_pred)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.9999988e-01
  1.2751899e-07]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  3.0988187e-12]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]
 [8.9630568e-03 7.8058237e-04 1.0332799e-02 3.9838660e-01 3.5165003e-01
  2.2988693e-01]]


In [13]:
# fashion mnist
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()